In [10]:
import torch
from torch import nn
from d2l import torch as d2l

In [11]:
#    X:输入  K：kernel
def corr2d(X,K):
    """计算二维互相关运算"""
    # 行数 列数
    h,w=K.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):# 遍历输出元素
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

In [12]:
X=torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K=torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)#验证二维互相关运算的输出

tensor([[19., 25.],
        [37., 43.]])

In [13]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [14]:
"""图像中目标的边缘检测"""
#0：黑色
#1：白色
# 构造一个6*8像素的黑白图像 中间4列为黑色0 其余像素为白色1
X=torch.ones((6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [15]:
# 构造一个高度为1 宽度为2的卷积核K 
# 如果水平相邻的两元素相同 输出为0  否则输出非0
K=torch.tensor([[1.0,-1.0]])

In [16]:
# 对参数X（输入）和K（卷积核执行互相关运算）
Y=corr2d(X,K)
Y
#输出Y中的1代表从白到黑色的边缘  -1戴白哦从黑色到白色的边缘
# 其他情况输出为0

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [17]:
# 把刚刚输入的二维图像转置 再进行互相关运算 
# 发现之前检测到的垂直边缘消失了
# 这个卷积核K只可以检测垂直边缘  无法检测水平边缘
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [18]:
# 如果我们只需寻找黑白边缘，那么以上[1, -1]的边缘检测器足以。
# 然而，当有了更复杂数值的卷积核，或者连续的卷积层时，我们不可能手动设计滤波器。
# 那么我们是否可以学习由X生成Y的卷积核呢？
# 现在让我们看看是否可以通过仅查看“输入-输出”对来学习由X生成Y的卷积核。 我们先构造一个卷积层，并将其卷积核初始化为随机张量。
# 接下来，在每次迭代中，我们比较Y与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。

# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))
lr=3e-2# 学习率


for i in range(10):# 循环执行10次
    Y_hat = conv2d(X)# 把X通过卷积层conv2d进行前向传播 得到预测输出Y_hat
    l = (Y_hat - Y) ** 2# 计算预测值Y_hat和Y之间的平方差  结果存在l中
    conv2d.zero_grad()# 梯度清零 以便重新进行梯度计算
    l.sum().backward()# 对损失l的总和反向传播计算梯度
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad# 用梯度下降法更新卷积核的权重
    if (i + 1) % 2 == 0:# 每迭代2次 输出当前损失值
        print(f'epoch {i+1}, loss {l.sum():.3f}')


epoch 2, loss 12.039
epoch 4, loss 3.131
epoch 6, loss 0.981
epoch 8, loss 0.351
epoch 10, loss 0.135


In [19]:
# 在10次迭代之后，误差已经降到足够低。现在我们来看看我们所学的卷积核的权重张量。

conv2d.weight.data.reshape((1,2))
# 和我们定义的卷积核K非常接近

tensor([[ 1.0230, -0.9487]])